In [ ]:
!git clone https://github.com/cknd/pyESN.git

Cloning into 'pyESN'...
remote: Enumerating objects: 106, done.
remote: Total 106 (delta 0), reused 0 (delta 0), pack-reused 106 (from 1)
Receiving objects: 100% (106/106), 1.17 MiB | 16.47 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [ ]:
!pip install pyESN

In [ ]:
!pip install pyswarm

  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4464 sha256=613105ef2a76cb4882a904cfe341ae029cd72337da8e80ec00cf6b436e164aa6
  Stored in directory: /root/.cache/pip/wheels/71/67/40/62fa158f497f942277cbab8199b05cb61c571ab324e67ad0d6
Successfully built pyswarm


In [ ]:
import sys
sys.path.append('/content/pyESN')

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from google.colab import drive
from datetime import datetime
from pyswarm import pso
from pyESN import ESN
# from pyESN.pyESN import ESN
from math import sqrt
import pandas as pd
import numpy as np

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wc_real/'

Mounted at /content/drive


In [ ]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [ ]:
filename = 'HV_data.csv'
HV_data = pd.read_csv(path + filename)

In [ ]:
set_index(HV_data)

In [ ]:
tm_data = HV_data
HV_data = HV_data.drop(columns=['HV'])
HV_data['HV'] = tm_data['HV']
HV_data

,"('GARCH', 'normal', 0)","('GARCH', 'gaussian', 0)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)","('FIGARCH', 'gaussian', 0)","('FIGARCH', 'gaussian', 1)",log_returns,"('GARCH', 'normal', 1)","('GARCH', 'gaussian', 1)","('GARCH', 'ged', 0)","('GARCH', 'ged', 1)",vix,unemployment_rate,usd_eur,usd_jpy,usd_gbp,usd_cny,usd_cad,gt_data,HV
Date,,,,,,,,,,,,,,,,,,,,
2000-02-02,-0.001864,-0.001864,-0.001782,-0.001782,-0.001782,-0.001782,0.000077,-0.001725,-0.001725,-0.001971,-0.001822,-5074.430765,-164.888872,-9.532198,-115483.935741,-25.204908,-667.139276,-20.309269,-53308.067923,-0.001870
2000-02-03,-0.001554,-0.001554,-0.001519,-0.001519,-0.001519,-0.001519,0.000020,-0.001557,-0.001557,-0.001745,-0.001612,-4871.204473,-152.205474,-8.790286,-107723.491598,-23.159152,-618.146493,-18.955134,-49391.829352,-0.001609
2000-02-04,-0.001080,-0.001080,-0.001076,-0.001076,-0.001076,-0.001076,0.000152,-0.001163,-0.001163,-0.001302,-0.001217,-3989.985998,-116.610418,-6.727301,-83922.602134,-17.659121,-475.895763,-14.738414,-38026.991031,-0.001164
2000-02-07,-0.000627,-0.000627,-0.000678,-0.000678,-0.000678,-0.000678,0.000090,-0.000738,-0.000738,-0.000863,-0.000784,-2979.521889,-78.602644,-4.500745,-57854.216936,-11.749618,-322.833213,-10.139957,-25799.214705,-0.000746
2000-02-08,-0.000202,-0.000202,-0.000311,-0.000311,-0.000311,-0.000311,-0.000335,-0.000335,-0.000335,-0.000364,-0.000368,-1886.674367,-43.682933,-2.468626,-33721.478802,-6.356898,-182.858321,-5.895064,-14616.177002,-0.000358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,-0.000358,-0.000358,-0.000690,-0.000690,-0.000690,-0.000690,-0.000161,-0.000493,-0.000493,-0.001023,-0.000502,-118.886061,65.925473,4.906428,75042.327304,7.151391,185.257251,6.351472,16.790559,0.000494
2024-08-27,-0.000694,-0.000694,-0.000899,-0.000899,-0.000899,-0.000899,-0.000193,-0.000707,-0.000707,-0.001244,-0.000717,-381.296847,28.527492,2.221002,29914.845185,3.476358,76.788614,2.415067,-3946.268810,0.000014
2024-08-28,-0.000954,-0.000954,-0.001012,-0.001012,-0.001012,-0.001012,-0.000154,-0.000846,-0.000846,-0.001311,-0.000852,-746.562442,-16.834312,-1.051394,-23800.733155,-1.072689,-53.866326,-2.260864,-8749.025243,-0.000477


# Split train, validation and test data

In [ ]:
# def rolling_window_transform(data, in_sample_size, out_of_sample_size):
#   total_size = in_sample_size + out_of_sample_size
#   in_sample = []
#   out_of_sample = []

#   for start in range(len(data) - total_size + 1):
#     in_sample.append(data.iloc[start : start + in_sample_size])
#     out_of_sample.append(data.iloc[start + in_sample_size : start + total_size])

#   return np.array(in_sample), np.array(out_of_sample)

# in_sample_size = 100
# out_of_sample_size = 5

# train_in_sample, train_out_of_sample = rolling_window_transform(train_data, in_sample_size, out_of_sample_size)
# val_in_sample, val_out_of_sample = rolling_window_transform(val_data, in_sample_size, out_of_sample_size)
# test_in_sample, test_out_of_sample = rolling_window_transform(test_data, in_sample_size, out_of_sample_size)

In [ ]:
# train_end = datetime(2020, 6, 30)
# test_start = datetime(2020, 10, 1)

In [ ]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  X_train = train_data.drop(columns=[target])
  y_train = train_data[target]

  X_val = val_data.drop(columns=[target])
  y_val = val_data[target]

  X_test = test_data.drop(columns=[target])
  y_test = test_data[target]

  return np.array(X_train), np.array(y_train), np.array(X_val), np.array(y_val), np.array(X_test), np.array(y_test)

# Model

In [ ]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## HV

In [ ]:
HV_X_train, HV_y_train, HV_X_val, HV_y_val, HV_X_test, HV_y_test = split_data(HV_data, 'HV')

In [ ]:
def esn_model(params, X_train, y_train, X_val):
  n_reservoir, spectral_radius, sparsity = int(params[0]), params[1], params[2]
  n_inputs = X_train.shape[1]
  n_outputs = 1

  esn = ESN(n_inputs=n_inputs, n_reservoir=n_reservoir, n_outputs=n_outputs, spectral_radius=spectral_radius, sparsity=sparsity)
  esn.fit(X_train, y_train)

  # Predict on validation data (for validation performance evaluation)
  pred_val = esn.predict(X_val)

  return pred_val

In [ ]:
def pso_objective(params):
  val_pred = esn_model(params, HV_X_train, HV_y_train, HV_X_val)
  loss = mean_squared_error(HV_y_val, val_pred)

  return loss

In [ ]:
# Initialize bounds for the parameters
lb = [10, 0.1, 0.01]  # Lower bounds for n_reservoir, spectral_radius, sparsity
ub = [500, 1.0, 0.5]  # Upper bounds for n_reservoir, spectral_radius, sparsity
swarmsize = 50
maxiter = 100

In [ ]:
optimal_params, _ = pso(pso_objective, lb, ub, swarmsize=swarmsize, maxiter=maxiter)

Stopping search: Swarm best objective change less than 1e-08


In [ ]:
optimal_params

array([4.70900456e+02, 1.00000000e+00, 8.13866392e-02])

In [ ]:
# optimal_params = [10. , 0.2336727 , 0.25712506]

In [ ]:
# Train and predict with optimal parameters
HV_test_predictions = esn_model(optimal_params, HV_X_train, HV_y_train, HV_X_test)

In [ ]:
HV_test_predictions.shape

(567, 1)

In [ ]:
HV_y_test.shape

(567,)

In [ ]:
mse = mean_squared_error(HV_y_test, HV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 1.5652154227060817e-06
RMSE: 0.0012510856975867328
MAE: 0.0005482640955077629
MAPE: 10189.688235318938%


In [ ]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/HV/'

# df_pred = pd.DataFrame(HV_test_predictions, columns=['esn_pred'])
# filename = 'ESN_pred.csv'
# df_pred.to_csv(path_1 + filename)